In [1]:
IMAGE_DIMS = (50,34,50,1)
# IMAGE_DIMS = (112,76,112,1)
OUTPUT_DIR = '../../../output/kaggle-bowl/step10/'
INPUT_DIR = '../../../input/step5-50/'
# INPUT_DIR = '../../../input/step5-112/'
DATASET_NAME_SUFFIX = '-centered-rotated'
LOAD_MODEL_FILE = None
#LOAD_MODEL_FILE = OUTPUT_DIR + 'tf-checkpoint-best5556'

In [2]:
import csv
import h5py
import numpy as np # linear algebra
import os
import logging
import tflearn

from modules.logging import logger
import modules.logging
import modules.lungprepare as lungprepare
import modules.utils as utils
import modules.cnn as cnn
from modules.utils import Timer

In [3]:
def start_training(model, input_dir, dataset_name_suffix, image_dims, output_dir):

    utils.mkdirs(output_dir, recreate=False)

    modules.logging.setup_file_logger(output_dir + 'out.log')
    
    dataset_path = utils.dataset_path(input_dir, 'train' + dataset_name_suffix, IMAGE_DIMS)
    with h5py.File(dataset_path, 'r') as train_hdf5:
        X = train_hdf5['X']
        Y = train_hdf5['Y']
        logger.info('X shape ' + str(X.shape))
        logger.info('Y shape ' + str(Y.shape))

        dataset_path = utils.dataset_path(input_dir, 'validate' + dataset_name_suffix, image_dims)
        with h5py.File(dataset_path, 'r') as validate_hdf5:
            X_validate = validate_hdf5['X']
            Y_validate = validate_hdf5['Y']
            logger.info('X_validate shape ' + str(X_validate.shape))
            logger.info('Y_validate shape ' + str(Y_validate.shape))

            logger.info('Starting CNN training...')
            model.fit(X, Y, validation_set=(X_validate, Y_validate), 
                      shuffle=True, batch_size=50, n_epoch=40,
                      show_metric=True, 
                      run_id='simplest1-'+str(image_dims))

In [4]:
logger.info('Prepare CNN for training')
#network = cnn.net_simplest1(IMAGE_DIMS)
#network = cnn.net_deepmedic_simple(IMAGE_DIMS)
network = cnn.net_alzheimer_cnn(IMAGE_DIMS)
model = cnn.prepare_cnn_model(network, OUTPUT_DIR, model_file=None)

2017-03-08 12:03:28,962 INFO Prepare CNN for training
2017-03-08 12:03:29,040 INFO Prepare CNN
2017-03-08 12:03:29,041 INFO Preparing output dir
2017-03-08 12:03:29,042 INFO Initializing network...


In [5]:
logger.info('Train CNN')
start_training(model, INPUT_DIR, DATASET_NAME_SUFFIX, IMAGE_DIMS, OUTPUT_DIR)
model.save(OUTPUT_DIR + 'final')
logger.info('==== ALL DONE ====')

Training Step: 439  | total loss: 0.22516 | time: 10.819s
| Adam | epoch: 040 | loss: 0.22516 - acc: 0.9300 -- iter: 500/538
Training Step: 440  | total loss: 0.22198 | time: 12.903s
| Adam | epoch: 040 | loss: 0.22198 - acc: 0.9270 | val_loss: 1.43148 - val_acc: 0.4000 -- iter: 538/538
--
INFO:tensorflow:/notebooks/output/kaggle-bowl/step10/tf-checkpoint-440 is not in all_model_checkpoint_paths. Manually adding it.


2017-03-08 12:12:16,460 INFO /notebooks/output/kaggle-bowl/step10/tf-checkpoint-440 is not in all_model_checkpoint_paths. Manually adding it.


INFO:tensorflow:/notebooks/output/kaggle-bowl/step10/final is not in all_model_checkpoint_paths. Manually adding it.


2017-03-08 12:12:16,912 INFO /notebooks/output/kaggle-bowl/step10/final is not in all_model_checkpoint_paths. Manually adding it.
2017-03-08 12:12:16,992 INFO ==== ALL DONE ====


In [6]:
logger.info('Evaluate model from dataset')
dataset_path = utils.dataset_path(INPUT_DIR, 'validate' + DATASET_NAME_SUFFIX, IMAGE_DIMS)
cnn.evaluate_dataset(dataset_path, model)

2017-03-08 12:12:16,998 INFO Evaluate model from dataset
2017-03-08 12:12:16,999 DEBUG X_test shape (90, 50, 34, 50, 1)
2017-03-08 12:12:17,001 DEBUG Y_test shape (90, 2)
2017-03-08 12:12:17,001 INFO Evaluate performance on dataset ../../../input/step5-50/validate-centered-rotated-50-34-50.h5...
2017-03-08 12:12:18,004 INFO Accuracy: [0.40000001192092893]
